In [ ]:
#Abandon all hope ye who enter here

In [ ]:
#mass model trainer

def model_trainer(X_train, y_train, X_test, y_test, random_seed):
    stratify = 0 #switch whether or not to stratify target_variable
    target_variable = "bank_account" #maybe we use this for the next project too
    niter = 100 #number iterators
    num_cv = 5 #num cross validation folds
    verbose=0 #quiet 0 1 2 loud
    num_jobs = -1 #-1 all cpu cores, 1 disables parallelization, 2 uses specified number
    scoring = "f1" #if we want to change scoring for whatever reason
    #todo: random_train_split make it so you randomize splits between models

 
    #not random parameters for logistic
    target_variable = "bank_account" #maybe we use this for the next project too
    niter_logi = 50 #number iterators
    num_cv_logi = 20 #num cross validation folds
    verbose=0 #quiet 0 1 2 loud
    num_jobs = -1 #-1 all cpu cores, 1 disables parallelization, 2 uses specified number
    scoring_logi = "f1" #if we want to change scoring for whatever reason
    random_seed = 42
    penaltea = ['l2', 'l1'] #penalty for logistic regression
    C_log = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] #C for logistic regression
    slogver = [ 'SGD', 'liblinear'] #solver for logistic regression
    miter = [120, 130, 135, 140, 145, 150] #max iterations for logistic regression
    logi_weights = { 0 : 0.1, 1: 0.9}

    #not random parameters for random forest
    niter_forest = 100
    forest_cv = 15
    scoring_forest = "f1"
    nestimators = [10, 25, 55] # num estimators for random forest
    big_deep = [15, 25, 30] #maximum depth for random forest
    min_forest_split = [4, 5, 6, 10, 12] # min_samples split
    min_tea_leafs = [3, 4, 5, 10, 15] #min_samples_leafs
    bootstrap = [True, False] # boostrap
    cryterio = ['gini', 'entropy'] # criterion
    forest_weights = { 0 : 0.5, 1: 0.5}

    #not random params for random xgboost
    xbgeta = [0.2, 0.3, 0.5, 0.75, 1] #eta
    XgammaBOOST = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  #gamma 
    deep_boost = [5, 6, 7, 8] #max_depth
    min_boost_weight = [2, 3, 4, 5] #min_child_weight
    xgboosamplet = [0.5, 0.6, 0.7, 0.8, 0.9] #subsample
    xgbytree = [0.6, 0.8, 1.0] #colsample_bytree
    xgboojective = ["binary:logistic"] #objective
    xgauc = ["auc"] #eval_metric
    xgscale = [1.2, 1.3] #scale_pos_weight
    xgbniter = 50
    xgbcv = 15
    scoring_xgb = "roc_auc"
    
    #not random params for svm
    svm_c = [0.1, 1, 10, 100]
    svm_kernel = ["linear", "rbf", "poly"]
    svm_gamma = ["scale", "auto"]
    svm_degree = [3, 4, 5]
    svc_niter = 10
    svc_cv = 5
    svc_weight = {0 : 1, 1: 5}

    #voting classifier
    voting = "hard" #can be soft


    #print("shape: ", target.shape)
    #print("unique values: ", target.unique())
    #print(train.dtypes)
    #print(target.dtypes)
    #print(target)

    #Darth Loger
    #train logistic
    #high recall, low precision
    tuned_logistic = LogisticRegression(class_weight=logi_weights)

    params = {
        'penalty': penaltea, 
        'C': C_log,
        'solver': slogver,
        'max_iter': miter
    }
    tuned_logistic = RandomizedSearchCV(estimator=tuned_logistic, 
                                        param_distributions=params, 
                                        n_iter=niter_logi, cv=num_cv_logi, 
                                        random_state=random_seed, verbose=verbose,
                                        n_jobs = num_jobs,
                                        scoring=scoring_logi)
    tuned_logistic.fit(X_train, y_train)
    print(tuned_logistic.best_params_)
    tuned_logistic = tuned_logistic.best_estimator_

    #Darth Lorax
    #train random forest
    #0.7 precision 0.3 recall
    tuned_forest = RandomForestClassifier(class_weight=forest_weights)

    params = {
    'n_estimators': nestimators, 
    'max_depth': big_deep,
    'min_samples_split': min_forest_split,
    'min_samples_leaf': min_tea_leafs,
    'bootstrap': bootstrap,
    'criterion': cryterio
    }

    tuned_forest = RandomizedSearchCV(estimator=tuned_forest,
                                        param_distributions=params,
                                        n_iter=niter_forest, cv=forest_cv,
                                        random_state=random_seed, verbose=verbose,
                                        n_jobs = num_jobs,
                                        scoring=scoring_forest)
    tuned_forest.fit(X_train, y_train)
    print(tuned_forest.best_params_)
    tuned_forest = tuned_forest.best_estimator_

    #Darth XGBious
    #train xgboost
    # 0.66 precision 0.33 recall
    tuned_xgboost = xgb.XGBClassifier()

    params = {
        "eta" : xbgeta,
        "gamma": XgammaBOOST,
        "max_depth" : deep_boost,
        "min_child_weight" : min_boost_weight,
        "subsample" : xgboosamplet,
        "colsample_bytree" : xgbytree,
        "objective" : xgboojective,
        "eval_metric" : xgauc,
        "scale_pos_weight" : xgscale
    }
    tuned_xgboost = RandomizedSearchCV(estimator=tuned_xgboost,
                                        param_distributions=params,
                                        n_iter=xgbniter, cv=xgbcv,
                                        random_state=random_seed, verbose=verbose,
                                        n_jobs = num_jobs,
                                        scoring=scoring_xgb)
    tuned_xgboost.fit(X_train, y_train)
    print(tuned_xgboost.best_params_)
    tuned_xgboost = tuned_xgboost.best_estimator_


    #SVM
    
    params = {
        "C" : svm_c,
        "kernel" : svm_kernel,
        "gamma" : svm_gamma,
        "degree" : svm_degree
    }
    tuned_svc = SVC(class_weight = svc_weight)
    tuned_svc = RandomizedSearchCV( estimator=tuned_svc,
                                    param_distributions=params,
                                    n_iter = svc_niter,
                                    cv = svc_cv,
                                    random_state=random_seed
    )
    tuned_svc.fit(X_train, y_train)
    print(tuned_svc.best_params_)
    tuned_svc = tuned_svc.best_estimator_

    #classifier
    sith_council = VotingClassifier(estimators = [
                                    ("logistic", tuned_logistic),
                                    ("random_forest", tuned_forest),
                                    ("xgboost", tuned_xgboost),
                                    ("svm", tuned_svc)], voting=voting
    )
    sith_council.fit(X_train, y_train)


    #The great wall of prints

    print("----------")
    print("Logistic Regression")
    print("----------")
    print("Classification report on train data:")
    print(classification_report(y_train, tuned_logistic.predict(X_train)))
    print("----------")
    print("Classification report on test data:")
    print(classification_report(y_test, tuned_logistic.predict(X_test)))
    print("----------")
    print("Random Forest")
    print("----------")
    print("Classification report on train data:")
    print(classification_report(y_train, tuned_forest.predict(X_train)))
    print("----------")
    print("Classification report on test data:")
    print(classification_report(y_test, tuned_forest.predict(X_test)))
    print("----------")
    print("XGBoost")
    print("----------")
    print("Classification report on train data:")
    print(classification_report(y_train, tuned_xgboost.predict(X_train)))
    print("----------")
    print("Classification report on test data:")
    print(classification_report(y_test, tuned_xgboost.predict(X_test)))
    print("----------")
    print("SVM")
    print("----------")
    print("Classification report on train data:")
    print(classification_report(y_train, tuned_svc.predict(X_train)))
    print("----------")
    print("Classification report on test data:")
    print(classification_report(y_test, tuned_svc.predict(X_test)))
    print("----------")
    print("Voting Classifier")
    print("----------")
    print("Classification report on train data:")
    print(classification_report(y_train, sith_council.predict(X_train)))
    print("----------")
    print("Classification report on test data:")
    print(classification_report(y_test, sith_council.predict(X_test)))
    print("----------")


In [ ]:
#mess with params for logi 

target_variable = "bank_account" #maybe we use this for the next project too
niter_logi = 50 #number iterators
num_cv_logi = 20 #num cross validation folds
verbose=0 #quiet 0 1 2 loud
num_jobs = -1 #-1 all cpu cores, 1 disables parallelization, 2 uses specified number
scoring_logi = "f1_macro" #if we want to change scoring for whatever reason
random_seed = 42
penaltea = ['l2', 'l1'] #penalty for logistic regression
C_log = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] #C for logistic regression
slogver = [ 'SGD', 'liblinear'] #solver for logistic regression
miter = [120, 130, 135, 140, 145, 150] #max iterations for logistic regression
logi_weights = { 0 : 0.4, 1: 0.6}

tuned_logistic = LogisticRegression(class_weight=logi_weights)

params = {
    'penalty': penaltea, 
    'C': C_log,
    'solver': slogver,
    'max_iter': miter
}

tuned_logistic = RandomizedSearchCV(estimator=tuned_logistic, 
                                    param_distributions=params, 
                                    n_iter=niter_logi, cv=num_cv_logi, 
                                    random_state=random_seed, verbose=verbose,
                                    n_jobs = num_jobs,
                                    scoring=scoring_logi)
tuned_logistic.fit(X_train_balanced, y_train_balanced)
best_params = tuned_logistic.best_params_
tuned_logistic = tuned_logistic.best_estimator_
print("----------")
print("Logistic Regression")
print("----------")
print("Classification report on train data:")
print(classification_report(y_train_balanced, tuned_logistic.predict(X_train_balanced)))
print("----------")
print("----------")
print("Classification report on imba train data:")
print(classification_report(y_train, tuned_logistic.predict(X_train)))
print("----------")
print("Classification report on test data:")
print(classification_report(y_test, tuned_logistic.predict(X_test)))
print("----------")
print(best_params)

In [ ]:
#mess with params for forest

#not random parameters for random forest
niter_forest = 100
forest_cv = 15
scoring_forest = "f1"
nestimators = [50, 100, 150, 200, 250, 300] # num estimators for random forest
big_deep = [8, 9, 10, 11, 12, 13, 14, 15] #maximum depth for random forest
min_forest_split = [3, 4, 5, 6, 10, 12] # min_samples split
min_tea_leafs = [3, 4, 5, 10, 15] #min_samples_leafs
bootstrap = [True, False] # boostrap
cryterio = ['gini', 'entropy'] # criterion
forest_weights = { 0 : 0.6, 1: 0.4}

tuned_forest = RandomForestClassifier(class_weight=forest_weights)

params = {
'n_estimators': nestimators, 
'max_depth': big_deep,
'min_samples_split': min_forest_split,
'min_samples_leaf': min_tea_leafs,
'bootstrap': bootstrap,
'criterion': cryterio
}

tuned_forest = RandomizedSearchCV(estimator=tuned_forest,
                                    param_distributions=params,
                                    n_iter=niter_forest, cv=forest_cv,
                                    random_state=random_seed, verbose=verbose,
                                    n_jobs = num_jobs,
                                    scoring=scoring_forest)
tuned_forest.fit(X_train_balanced, y_train_balanced)
best_params = tuned_forest.best_params_
tuned_forest = tuned_forest.best_estimator_
print("----------")
print("Forest Random")
print("----------")
print("Classification report on train data:")
print(classification_report(y_train_balanced, tuned_forest.predict(X_train_balanced)))
print("----------")
print("Classification report on imba train data:")
print(classification_report(y_train, tuned_forest.predict(X_train)))
print("----------")
print("Classification report on test data:")
print(classification_report(y_test, tuned_forest.predict(X_test)))
print("----------")
print(best_params)


In [ ]:
#not random params for random xgboost
xbgeta = [0.05, 0.1, 0.15] #eta
XgammaBOOST = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  #gamma 
xgb_num_boost= [50, 100] #num iter
deep_boost = [3, 4, 5, 6, 7, 8] #max_depth
min_boost_weight = [2, 3, 4, 5] #min_child_weight
xgboosamplet = [0.5, 0.6, 0.7, 0.8, 0.9] #subsample
xgbytree = [0.6, 0.8, 1.0] #colsample_bytree
xgboojective = ["binary:logistic"] #objective
xgauc = ["auc"] #eval_metric
xgscale = [1.0] #scale_pos_weight
xgbcv = 15
scoring_xgb = "roc_auc"

#mess with params for xgboost
tuned_xgboost = xgb.XGBClassifier()

params = {
    "eta" : xbgeta,
    "gamma": XgammaBOOST,
    "max_depth" : deep_boost,
    "min_child_weight" : min_boost_weight,
    "subsample" : xgboosamplet,
    "colsample_bytree" : xgbytree,
    "objective" : xgboojective,
    "eval_metric" : xgauc,
    "scale_pos_weight" : xgscale,
    'n_estimators' : xgb_num_boost
}
tuned_xgboost = RandomizedSearchCV(estimator=tuned_xgboost,
                                    param_distributions=params, cv=xgbcv,
                                    random_state=random_seed, verbose=verbose,
                                    n_jobs = num_jobs,
                                    scoring=scoring_xgb)
tuned_xgboost.fit(X_train_balanced, y_train_balanced)
best_params = tuned_xgboost.best_params_
tuned_xgboost = tuned_xgboost.best_estimator_
print("----------")
print("gxboost")
print("----------")
print("Classification report on train data:")
print(classification_report(y_train_balanced, tuned_xgboost.predict(X_train_balanced)))
print("----------")
print("Classification report on imba data:")
print(classification_report(y_train, tuned_xgboost.predict(X_train)))
print("----------")
print("Classification report on test data:")
print(classification_report(y_test, tuned_xgboost.predict(X_test)))
print("----------")
print(best_params)

In [ ]:
#mess with svm
svm_c = [0.1, 0.5, 0.75, 1, 2, 3, 4]
svm_kernel = ["linear", "rbf"]
svm_gamma = ["scale", "auto"]
svc_niter = 10
svc_cv = 15
svc_weight = {0 : 0.9, 1: 1.3}

params = {
    "C" : svm_c,
    "kernel" : svm_kernel,
    "gamma" : svm_gamma
}
tuned_svc = SVC(class_weight = svc_weight)
tuned_svc = RandomizedSearchCV( estimator=tuned_svc,
                                param_distributions=params,
                                n_iter = svc_niter,
                                cv = svc_cv,
                                random_state=random_seed
)
tuned_svc.fit(X_train_balanced, y_train_balanced)
best_params = tuned_svc.best_params_
tuned_svc = tuned_svc.best_estimator_
print("----------")
print("svn")
print("----------")
print("Classification report on train data:")
print(classification_report(y_train_balanced, tuned_svc.predict(X_train_balanced)))
print("----------")
print("Classification report on imba data:")
print(classification_report(y_train, tuned_svc.predict(X_train)))
print("----------")
print("Classification report on test data:")
print(classification_report(y_test, tuned_svc.predict(X_test)))
print("----------")
print(best_params)
